In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.utils import save_image
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

# Configuration
IMG_HEIGHT = 128
IMG_WIDTH = 128
CHANNELS = 3
BATCH_SIZE = 8
LATENT_DIM = 128
EPOCHS = 5000
ADA_TARGET = 0.6
ADA_INTERVAL = 4
ADA_SPEED = 0.01
ADA_MAX_P = 0.85  # Clamp augmentation probability
FID_INTERVAL = 100  # Check FID every 100 batches
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class AnimalDataset(Dataset):
    def __init__(self, root_dir, class_name = "", transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        
        if class_name == "":
            for class_dir in os.listdir(root_dir):
                class_path = os.path.join(root_dir, class_dir)
                if os.path.isdir(class_path):
                    for img_name in os.listdir(class_path):
                        self.image_paths.append(os.path.join(class_path, img_name))
        else:
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    self.image_paths.append(os.path.join(class_path, img_name))
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image

# Basic transforms (ADA will handle additional augmentations)
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize to [-1, 1]
])

# Create dataset and dataloader
dataset = AnimalDataset(root_dir='Newdata/Train', class_name="", transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

print(len(dataset))

82


In [3]:
class AdaptiveAugment:
    def __init__(self, ada_target=0.6, ada_speed=0.01, max_p=0.8, warmup=500):
        """
        Args:
            ada_target: Target discriminator accuracy (default: 0.6)
            ada_speed: How quickly p adjusts (smaller = more stable)
            max_p: Maximum augmentation probability (recommended ≤0.85)
            warmup: Batches before allowing p to increase (prevents early over-augmentation)
        """
        self.ada_target = ada_target
        self.ada_speed = ada_speed
        self.max_p = max_p
        self.warmup = warmup
        self.augment_p = 0
        self.r_t = 0  # Running accuracy estimate
        self.steps = 0
        self.sign_history = []

    def update(self, real_pred):
        """Update augmentation probability based on discriminator accuracy"""
        self.steps += 1
        
        # Calculate current discriminator accuracy
        current_acc = real_pred.sigmoid().mean().item()
        self.r_t = 0.9 * self.r_t + 0.1 * current_acc  # EMA smoothing

        # Determine adjustment direction
        sign = 1 if self.r_t > self.ada_target else -1
        self.sign_history.append(sign)

        # Only adjust p if:
        # 1. Past warmup period AND
        # 2. Consistent direction for 5 batches
        if self.steps > self.warmup and \
           len(self.sign_history) > 5 and \
           sum(self.sign_history[-5:]) == 5 * sign:
            
            # More conservative adjustment when p is high
            effective_speed = self.ada_speed * (1 - 0.5 * (self.augment_p / self.max_p))
            
            self.augment_p += sign * effective_speed
            self.augment_p = min(max(self.augment_p, 0), self.max_p)  # Clamp to [0, max_p]

        return self.augment_p

    def apply_augment(self, x, p=None):
        """Apply augmentations with probability p (uses current p if None)"""
        p = self.augment_p if p is None else min(p, self.max_p)
        if p <= 0:
            return x
        
        # Apply augmentations with probability p
        if torch.rand(1, device=x.device).item() < p:
            # Random horizontal flip (50% chance)
            if torch.rand(1, device=x.device).item() > 0.5:
                x = torch.flip(x, [3])
            
            # Smaller rotation range for stability (-10° to 10°)
            angle = torch.empty(1, device=x.device).uniform_(-10, 10).item()
            x = transforms.functional.rotate(x, angle)
            
            # More subtle color jitter
            brightness = torch.empty(1, device=x.device).uniform_(0.9, 1.1).item()
            contrast = torch.empty(1, device=x.device).uniform_(0.9, 1.1).item()
            x = transforms.functional.adjust_brightness(x, brightness)
            x = transforms.functional.adjust_contrast(x, contrast)
            
            # Smaller translation (2-3%)
            translate_x = torch.empty(1, device=x.device).uniform_(-0.03, 0.03).item()
            translate_y = torch.empty(1, device=x.device).uniform_(-0.03, 0.03).item()
            x = transforms.functional.affine(
                x, angle=0, translate=[translate_x, translate_y], 
                scale=1, shear=0
            )
        
        return x

from pytorch_fid import fid_score
import tempfile
import os
def get_samples_from_dataloader(dataloader, num_samples):
    """Properly extracts samples from a DataLoader"""
    samples = []
    total_collected = 0
    
    # Iterate through batches
    for batch in dataloader:
        # Handle batched data (batch could be tensors or dicts)
        if isinstance(batch, (torch.Tensor, np.ndarray)):
            batch_samples = batch
        elif isinstance(batch, dict):
            batch_samples = batch['images']  # Adjust based on your dataset
        else:
            batch_samples = batch[0]  # Common for (data, label) tuples
        
        # Add samples until we have enough
        remaining = num_samples - total_collected
        if remaining <= 0:
            break
            
        samples.append(batch_samples[:remaining])
        total_collected += batch_samples.shape[0]
    
    return torch.cat(samples)[:num_samples]  # Ensures exact number
def calculate_fid(generator, dataloader, device, num_samples=500):
    with tempfile.TemporaryDirectory() as real_dir, tempfile.TemporaryDirectory() as fake_dir:
        # 1. Save REAL images (correct DataLoader handling)
        real_images = get_samples_from_dataloader(dataloader, num_samples)
        for i, img in enumerate(real_images):
            img_pil = transforms.ToPILImage()(img.cpu())
            img_pil.save(os.path.join(real_dir, f'real_{i}.png'))
        
        # 2. Generate FAKE images
        with torch.no_grad():
            for i in range(0, num_samples, BATCH_SIZE):
                batch_size = min(BATCH_SIZE, num_samples - i)
                z = torch.randn(batch_size, LATENT_DIM, device=device)
                fake_batch = generator(z).cpu()
                
                for j in range(batch_size):
                    transforms.ToPILImage()(fake_batch[j]).save(
                        os.path.join(fake_dir, f'fake_{i+j}.png'))
        
        # 3. Compute FID
        return fid_score.calculate_fid_given_paths(
            [real_dir, fake_dir],
            batch_size=min(BATCH_SIZE, num_samples),
            device=device,
            dims=2048
        )

In [4]:
# Generator Network
class ResBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(channels, channels, 3, 1, 1),
            nn.BatchNorm2d(channels),
            nn.LeakyReLU(0.2),
            nn.Conv2d(channels, channels, 3, 1, 1),
        )
    def forward(self, x):
        return x + self.conv(x)  # Skip connection
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        
        self.init_size = IMG_HEIGHT // 8
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))
        
        self.conv_blocks = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            
            ResBlock(64),
            
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            
            ResBlock(32),
            
            nn.Conv2d(32, CHANNELS, 3, stride=1, padding=1),
            nn.BatchNorm2d(CHANNELS, 0.8),
            nn.Tanh()
        )
        # self.conv_blocks = nn.Sequential(
        #     nn.ConvTranspose2d(128, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
        #     nn.BatchNorm2d(128, 0.8),
        #     nn.LeakyReLU(0.2, inplace=True),
            
        #     nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
        #     nn.BatchNorm2d(64, 0.8),
        #     nn.LeakyReLU(0.2, inplace=True),
            
        #     nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
        #     nn.BatchNorm2d(32, 0.8),
        #     nn.LeakyReLU(0.2, inplace=True),
            
        #     nn.ConvTranspose2d(32, CHANNELS, kernel_size=3, stride=2, padding=1, output_padding=1),
        #     nn.BatchNorm2d(CHANNELS, 0.8),
        #     nn.LeakyReLU(0.2, inplace=True),
            
        #     nn.ConvTranspose2d(CHANNELS, CHANNELS, kernel_size=3, stride=1, padding=1),
        #     nn.BatchNorm2d(CHANNELS, 0.8),
        #     nn.Tanh()
        # )
    
    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, stride = 2, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=stride, padding=1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block
        
        self.model = nn.Sequential(
            *discriminator_block(CHANNELS, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128, stride=1),
            
        )
        
        # The height and width of downsampled image
        ds_size = IMG_HEIGHT // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
    
    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        return validity


In [5]:
# Initialize networks
generator = Generator(LATENT_DIM).to(DEVICE)
discriminator = Discriminator().to(DEVICE)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Loss function
adversarial_loss = nn.BCELoss()

# ADA module
ada = AdaptiveAugment(
    ada_target=0.65,  # Slightly higher target
    ada_speed=0.005,  # Slower adjustments
    max_p=0.8,        # Conservative maximum
    warmup=200       # Longer warmup for small datasets
)

In [ ]:
# Training
best_fid = 9999999
EPOCHS = 2000
for epoch in range(EPOCHS*0, 1*EPOCHS):
    generator.train()
    for i, imgs in enumerate(dataloader):
        # Configure input
        real_imgs = imgs.to(DEVICE)
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()
        
        # Sample noise as generator input
        z = torch.randn(imgs.shape[0], LATENT_DIM, device=DEVICE)
        
        # Generate a batch of images
        fake_imgs = generator(z)
        
        # Apply adaptive augmentation
        p = ada.augment_p
        real_aug = ada.apply_augment(real_imgs, p)
        fake_aug = ada.apply_augment(fake_imgs.detach(), p)
        
        # Real images
        real_pred = discriminator(real_aug)
        real_loss = adversarial_loss(real_pred, torch.ones(real_pred.size(), device=DEVICE))
        
        # Fake images
        fake_pred = discriminator(fake_aug)
        fake_loss = adversarial_loss(fake_pred, torch.zeros(fake_pred.size(), device=DEVICE))
        
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2
        
        d_loss.backward()
        optimizer_D.step()
        
        # Update ADA probability
        if i % ADA_INTERVAL == 0:
            p = ada.update(real_pred)
        
        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()
        
        # Generate a batch of images
        gen_imgs = generator(z)
        
        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), torch.ones(gen_imgs.size(0), 1, device=DEVICE))
        
        g_loss.backward()
        optimizer_G.step()
        
        # Print progress
        if i % 50 == 0:
            print(
                f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(dataloader)}] "
                f"[D loss: {d_loss.item():.4f}] [G loss: {g_loss.item():.4f}] [ADA p: {p:.3f}]"
            )
        # Update ADA with FID feedback (every fid_interval steps)
        
        # Apply augmentation
        real_aug = ada.apply_augment(real_imgs, p)
    # Save generated images every epoch
    if epoch % 10 == 9:
        with torch.no_grad():
            sample_z = torch.randn(8, LATENT_DIM, device=DEVICE)
            gen_imgs = generator(sample_z)
            save_image(gen_imgs.data, f"images_v4_resblock/{epoch}.png", nrow=4, normalize=True)
        
    if epoch % 50 == 49:
            generator.eval()
            current_fid = calculate_fid(generator, dataloader.dataset, DEVICE)
            print(f"Epoch {epoch} | FID: {current_fid:.2f} | ADA p: {p:.3f}")
            
            # Save best model
            if current_fid < best_fid:
                best_fid = current_fid
                torch.save(generator.state_dict(), "best_generator_res_v4.pth")
                
            if current_fid > 2 * best_fid:  # Divergence detected
                print("Stopping early - FID doubled")
                break

# Save models
torch.save(generator.state_dict(), "generator_res.pth")
torch.save(discriminator.state_dict(), "discriminator_res.pth")


[Epoch 0/2000] [Batch 0/11] [D loss: 0.6934] [G loss: 0.6912] [ADA p: 0.000]
[Epoch 1/2000] [Batch 0/11] [D loss: 0.6872] [G loss: 0.6969] [ADA p: 0.000]
[Epoch 2/2000] [Batch 0/11] [D loss: 0.6786] [G loss: 0.6812] [ADA p: 0.000]
[Epoch 3/2000] [Batch 0/11] [D loss: 0.6629] [G loss: 0.7232] [ADA p: 0.000]
[Epoch 4/2000] [Batch 0/11] [D loss: 0.5973] [G loss: 0.6644] [ADA p: 0.000]
[Epoch 5/2000] [Batch 0/11] [D loss: 0.6648] [G loss: 0.6236] [ADA p: 0.000]
[Epoch 6/2000] [Batch 0/11] [D loss: 0.4877] [G loss: 0.8754] [ADA p: 0.000]
[Epoch 7/2000] [Batch 0/11] [D loss: 0.6213] [G loss: 0.7166] [ADA p: 0.000]
[Epoch 8/2000] [Batch 0/11] [D loss: 0.5177] [G loss: 1.1297] [ADA p: 0.000]
[Epoch 9/2000] [Batch 0/11] [D loss: 0.4223] [G loss: 0.9126] [ADA p: 0.000]
[Epoch 10/2000] [Batch 0/11] [D loss: 0.3744] [G loss: 1.1268] [ADA p: 0.000]
[Epoch 11/2000] [Batch 0/11] [D loss: 0.4408] [G loss: 1.1330] [ADA p: 0.000]
[Epoch 12/2000] [Batch 0/11] [D loss: 0.3827] [G loss: 1.2556] [ADA p: 0.0

100%|██████████| 63/63 [00:11<00:00,  5.34it/s]


Epoch 49 | FID: 391.78 | ADA p: 0.000
[Epoch 50/2000] [Batch 0/11] [D loss: 1.4889] [G loss: 0.6463] [ADA p: 0.000]
[Epoch 51/2000] [Batch 0/11] [D loss: 0.4836] [G loss: 1.7216] [ADA p: 0.000]
[Epoch 52/2000] [Batch 0/11] [D loss: 0.4932] [G loss: 1.5185] [ADA p: 0.000]
[Epoch 53/2000] [Batch 0/11] [D loss: 0.1030] [G loss: 2.6371] [ADA p: 0.000]
[Epoch 54/2000] [Batch 0/11] [D loss: 0.4032] [G loss: 3.7463] [ADA p: 0.000]
[Epoch 55/2000] [Batch 0/11] [D loss: 0.5828] [G loss: 1.7398] [ADA p: 0.000]
[Epoch 56/2000] [Batch 0/11] [D loss: 0.2099] [G loss: 1.6861] [ADA p: 0.000]
[Epoch 57/2000] [Batch 0/11] [D loss: 0.1199] [G loss: 0.6187] [ADA p: 0.000]
[Epoch 58/2000] [Batch 0/11] [D loss: 0.3434] [G loss: 0.9703] [ADA p: 0.000]
[Epoch 59/2000] [Batch 0/11] [D loss: 0.3593] [G loss: 2.8891] [ADA p: 0.000]
[Epoch 60/2000] [Batch 0/11] [D loss: 0.1584] [G loss: 1.6314] [ADA p: 0.000]
[Epoch 61/2000] [Batch 0/11] [D loss: 0.6051] [G loss: 1.4583] [ADA p: 0.000]
[Epoch 62/2000] [Batch 0/1

100%|██████████| 63/63 [00:11<00:00,  5.49it/s]


Epoch 99 | FID: 393.70 | ADA p: 0.354
[Epoch 100/2000] [Batch 0/11] [D loss: 0.3240] [G loss: 1.4428] [ADA p: 0.354]
[Epoch 101/2000] [Batch 0/11] [D loss: 0.5549] [G loss: 1.0672] [ADA p: 0.354]
[Epoch 102/2000] [Batch 0/11] [D loss: 0.7307] [G loss: 1.4890] [ADA p: 0.354]
[Epoch 103/2000] [Batch 0/11] [D loss: 0.6885] [G loss: 0.7758] [ADA p: 0.354]
[Epoch 104/2000] [Batch 0/11] [D loss: 0.4883] [G loss: 1.2099] [ADA p: 0.343]
[Epoch 105/2000] [Batch 0/11] [D loss: 0.6135] [G loss: 0.7581] [ADA p: 0.331]
[Epoch 106/2000] [Batch 0/11] [D loss: 0.3207] [G loss: 1.2475] [ADA p: 0.319]
[Epoch 107/2000] [Batch 0/11] [D loss: 0.5420] [G loss: 1.7690] [ADA p: 0.307]
[Epoch 108/2000] [Batch 0/11] [D loss: 0.4714] [G loss: 0.5721] [ADA p: 0.299]
[Epoch 109/2000] [Batch 0/11] [D loss: 0.5943] [G loss: 0.6970] [ADA p: 0.299]
[Epoch 110/2000] [Batch 0/11] [D loss: 0.4084] [G loss: 0.7254] [ADA p: 0.299]
[Epoch 111/2000] [Batch 0/11] [D loss: 1.1866] [G loss: 0.6871] [ADA p: 0.291]
[Epoch 112/200

...

Epoch 1349 | FID: 342.61 | ADA p: 0.800
[Epoch 1350/2000] [Batch 0/11] [D loss: 0.2642] [G loss: 1.3152] [ADA p: 0.800]
[Epoch 1351/2000] [Batch 0/11] [D loss: 0.3958] [G loss: 1.2071] [ADA p: 0.800]
[Epoch 1352/2000] [Batch 0/11] [D loss: 0.1804] [G loss: 1.3551] [ADA p: 0.800]
[Epoch 1353/2000] [Batch 0/11] [D loss: 0.7358] [G loss: 0.5403] [ADA p: 0.800]
[Epoch 1354/2000] [Batch 0/11] [D loss: 0.3835] [G loss: 0.7907] [ADA p: 0.800]
[Epoch 1355/2000] [Batch 0/11] [D loss: 0.2850] [G loss: 1.3684] [ADA p: 0.800]
[Epoch 1356/2000] [Batch 0/11] [D loss: 0.1257] [G loss: 1.1914] [ADA p: 0.800]
[Epoch 1357/2000] [Batch 0/11] [D loss: 0.2641] [G loss: 1.8343] [ADA p: 0.800]
[Epoch 1358/2000] [Batch 0/11] [D loss: 0.3747] [G loss: 1.3769] [ADA p: 0.800]
[Epoch 1359/2000] [Batch 0/11] [D loss: 0.3637] [G loss: 2.0359] [ADA p: 0.800]
[Epoch 1360/2000] [Batch 0/11] [D loss: 0.5389] [G loss: 0.5108] [ADA p: 0.800]
[Epoch 1361/2000] [Batch 0/11] [D loss: 0.1970] [G loss: 1.0452] [ADA p: 0.800]


100%|██████████| 63/63 [00:10<00:00,  5.74it/s]


Epoch 1399 | FID: 345.36 | ADA p: 0.800
[Epoch 1400/2000] [Batch 0/11] [D loss: 0.5279] [G loss: 1.1556] [ADA p: 0.800]
[Epoch 1401/2000] [Batch 0/11] [D loss: 0.2438] [G loss: 0.8658] [ADA p: 0.800]
[Epoch 1402/2000] [Batch 0/11] [D loss: 0.3358] [G loss: 1.2923] [ADA p: 0.800]
[Epoch 1403/2000] [Batch 0/11] [D loss: 0.5038] [G loss: 0.7516] [ADA p: 0.800]
[Epoch 1404/2000] [Batch 0/11] [D loss: 0.5573] [G loss: 0.9522] [ADA p: 0.800]
[Epoch 1405/2000] [Batch 0/11] [D loss: 0.4265] [G loss: 1.2679] [ADA p: 0.800]
[Epoch 1406/2000] [Batch 0/11] [D loss: 0.2365] [G loss: 1.1390] [ADA p: 0.800]
[Epoch 1407/2000] [Batch 0/11] [D loss: 0.3662] [G loss: 1.9840] [ADA p: 0.800]
[Epoch 1408/2000] [Batch 0/11] [D loss: 0.3575] [G loss: 2.2330] [ADA p: 0.800]
[Epoch 1409/2000] [Batch 0/11] [D loss: 0.2865] [G loss: 0.7714] [ADA p: 0.800]
[Epoch 1410/2000] [Batch 0/11] [D loss: 0.1397] [G loss: 1.4437] [ADA p: 0.800]
[Epoch 1411/2000] [Batch 0/11] [D loss: 0.2459] [G loss: 0.8673] [ADA p: 0.800]


100%|██████████| 63/63 [00:13<00:00,  4.76it/s]


Epoch 1449 | FID: 340.59 | ADA p: 0.800
[Epoch 1450/2000] [Batch 0/11] [D loss: 0.7798] [G loss: 1.6574] [ADA p: 0.800]
[Epoch 1451/2000] [Batch 0/11] [D loss: 0.1608] [G loss: 1.1958] [ADA p: 0.800]
[Epoch 1452/2000] [Batch 0/11] [D loss: 0.2082] [G loss: 1.2228] [ADA p: 0.800]
[Epoch 1453/2000] [Batch 0/11] [D loss: 0.3566] [G loss: 0.8619] [ADA p: 0.800]
[Epoch 1454/2000] [Batch 0/11] [D loss: 0.2901] [G loss: 1.8173] [ADA p: 0.800]
[Epoch 1455/2000] [Batch 0/11] [D loss: 0.3755] [G loss: 0.4558] [ADA p: 0.800]
[Epoch 1456/2000] [Batch 0/11] [D loss: 0.5408] [G loss: 1.0578] [ADA p: 0.800]
[Epoch 1457/2000] [Batch 0/11] [D loss: 0.4824] [G loss: 1.4261] [ADA p: 0.800]
[Epoch 1458/2000] [Batch 0/11] [D loss: 0.2700] [G loss: 1.6145] [ADA p: 0.800]
[Epoch 1459/2000] [Batch 0/11] [D loss: 0.5720] [G loss: 0.3694] [ADA p: 0.800]
[Epoch 1460/2000] [Batch 0/11] [D loss: 0.2898] [G loss: 1.4545] [ADA p: 0.800]
[Epoch 1461/2000] [Batch 0/11] [D loss: 0.2478] [G loss: 1.1522] [ADA p: 0.800]


In [ ]:
torch.cuda.is_available()

True